In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import entropy
import itertools, copy
from collections import defaultdict
import matplotlib as mpl
%matplotlib inline

In [3]:
def h(d):
    if isinstance(d, (int, float)):
        return entropy([d,1-d], base=2)
    else:
        return entropy(d, base=2)

In [4]:
def heterogeneity(theta,q=(0.9,0.1)):
    hz = h([1/2-(q[0]+q[1])/4, 
            1/2-(q[0]+q[1])/4, 
            ((1/2-theta[0])*q[0]+(1/2-theta[1])*q[1])/2, 
            ((1/2+theta[0])*q[0]+(1/2+theta[1])*q[1])/2])
    hzi = (h([(1-q[0])/2, (1-q[0])/2, (1/2-theta[0])*q[0], (1/2+theta[0])*q[0]])/2 + 
           h([(1-q[1])/2, (1-q[1])/2, (1/2-theta[1])*q[1], (1/2+theta[1])*q[1]])/2)
    return hz - hzi


In [5]:
def accuracy(p,theta,q=(0.9,0.1)):
    return sum([(
        (1-q[i])*((1-p[(0,0,i)])/2 + p[(0,1,i)]/2) + 
        q[i]*((1/2-theta[i])*(1-p[(1,0,i)])+(1/2+theta[i])*p[(1,1,i)])
    )/2 for i in [0,1]])

In [6]:
def local_constraint(p,theta,i,q=(0.9,0.1)):
    hyi = h((1-q[i])*(p[(0,0,i)]/2 + p[(0,1,i)]/2) + q[i]*((1/2-theta[i])*p[(1,0,i)] + (1/2+theta[i])*p[(1,1,i)]))
    hyai = (1-q[i])*h(p[(0,0,i)]/2 + p[(0,1,i)]/2) + q[i]*h((1/2-theta[i])*p[(1,0,i)] + (1/2+theta[i])*p[(1,1,i)])
    return hyi-hyai

In [7]:
def mutual_information(p,theta,q=(0.9,0.1)):
    pya0 = sum([(p[(0,0,i)]/2+p[0,1,i]/2)*(1-q[i])/(2-q[0]-q[1]) for i in [0,1]])
    pya1 = sum([((1/2-theta[i])*p[(1,0,i)]+(1/2+theta[i])*p[(1,1,i)])*q[i]/(q[0]+q[1]) for i in [0,1]])
    hyhat = h(pya0*(1-(q[0]+q[1])/2) + pya1*(q[0]+q[1])/2)
    hyhata = (1-(q[0]+q[1])/2)*h(pya0) + (q[0]+q[1])/2*h(pya1)
    return hyhat - hyhata